<a href="https://colab.research.google.com/github/graziaperna/NLP-project/blob/ML/MLPipeline.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [78]:
#!pip install -U spaCy
#!pip install keras
#!python -m spacy download it_core_news_lg
#!pip install -U jax jaxlib
#!pip install clean-text
#!pip install --upgrade ipykernel
#da eseguire al primo utilizzo

In [79]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [80]:
import string
import spacy
import json
import re
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer
from sklearn.base import TransformerMixin
from sklearn.pipeline import Pipeline
from spacy.lang.it.stop_words import STOP_WORDS
from spacy.lang.it import Italian
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.multioutput import MultiOutputClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, make_scorer
from sklearn.model_selection import GridSearchCV
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem.porter import PorterStemmer
from pandas import read_csv
from sklearn.svm import SVC
from sklearn import metrics
from sklearn import svm
from sklearn import neural_network
from sklearn import ensemble
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem.porter import PorterStemmer
from pandas import read_csv
import nltk


#init spaCy
punctuations = string.punctuation
nlp = spacy.load("it_core_news_lg")
stop_words = spacy.lang.it.stop_words.STOP_WORDS
parser = Italian()

# Custom transformer using spaCy
class predictors(TransformerMixin):
    def transform(self, X, **transform_params):
        # Cleaning Text
        return [clean_text(text) for text in X]

    def fit(self, X, y=None, **fit_params):
        return self

    def get_params(self, deep=True):
        return {}

# Basic function to clean the text
def clean_text(text):
    # Removing spaces and converting text into lowercase
    
    return text.strip().lower()

# Tokenizer function
def spacy_tokenizer(sentence):
    mytokens = parser(sentence)
    mytokens = [ word.text for word in mytokens ]
    # remove stop words
    mytokens = [ word for word in mytokens if word not in stop_words and word not in punctuations ]
    # return preprocessed list of tokens
    return mytokens

In [81]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [82]:
df = read_csv('/content/final_dataset_sexist.csv')
X = df['text']
y = df['label_sexist']
df.set_index('ID')

,text,label_sexist
ID,,
0,"In Nigeria, if you rape a woman, the men rape ...",not sexist
1,"Then, she's a keeper. 😉",not sexist
2,This is like the Metallica video where the poo...,not sexist
3,woman?,not sexist
4,I bet she wished she had a gun,not sexist
...,...,...
19995,girls really get fucked almost every weekend ?,not sexist
19996,The hatred for moslems are Reasonable and Just...,not sexist
19997,Now this is a woman who gets it. 👆,not sexist


In [83]:
vector = CountVectorizer(tokenizer = spacy_tokenizer, ngram_range=(1,1))

In [84]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.35, random_state=42)
y_test = y_test.reset_index(drop=True)

In [85]:
print(f"x_train shape: {X_train.shape}")  # (50, 4)
print(f"y_train shape: {y_train.shape}")  # (50, 4)  ==> 2d, so something is wrong
print(f"x_test shape: {X_test.shape}")  # (50,) => also bad
print(f"y_test shape: {y_test.shape}")


x_train shape: (13000,)
y_train shape: (13000,)
x_test shape: (7000,)
y_test shape: (7000,)


In [86]:
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.multiclass import OneVsRestClassifier
from sklearn.svm import LinearSVC

#Linear Support Vector Machine classifier
classifier = svm.LinearSVC()
# Multi-layer Perceptron classifier
#classifier = neural_network.MLPClassifier(hidden_layer_sizes=(512,),verbose=True,max_iter=10)
# Random Foreset
#classifier = ensemble.RandomForestClassifier()
# Create the pipeline
pipe = Pipeline([("cleaner", predictors()),
('vectorizer', vector),
('classifier', classifier)])

# Addestramento del modello
pipe.fit(X_train, y_train)

/usr/local/lib/python3.10/dist-packages/sklearn/feature_extraction/text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/svm/_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


Pipeline(steps=[('cleaner', <__main__.predictors object at 0x7f79acd48b50>),
                ('vectorizer',
                 CountVectorizer(tokenizer=<function spacy_tokenizer at 0x7f79b53a5870>)),
                ('classifier', LinearSVC())])

In [87]:
single_instance = ["you are my beautiful bitch "]

In [88]:
prediction = pipe.predict(single_instance)

In [89]:
print(f"Prevista classe: {prediction}")

Prevista classe: ['sexist']


In [90]:
#!pip install scikit-learn -U
import sklearn.metrics

In [91]:
predicted = pipe.predict(X_test)
sklearn.metrics.f1_score(y_test, predicted,average='weighted',zero_division=1)

0.7969008903746199

In [92]:
predicted = pipe.predict(X_test)

acc = metrics.accuracy_score(y_test, predicted)
print("Accuracy:", acc)
print("======================================================")
precision_macro = metrics.precision_score(y_test, predicted, average='macro')
recall_macro = metrics.recall_score(y_test, predicted, average='macro')
fm_macro = metrics.f1_score(y_test, predicted, average='macro')
print("P-macro={0}, R-macro={1}, F1-macro={2}".format(precision_macro, recall_macro, fm_macro))
print("======================================================")
precision_micro = metrics.precision_score(y_test, predicted, average='micro')
recall_micro = metrics.recall_score(y_test, predicted, average='micro')
fm_micro = metrics.f1_score(y_test, predicted, average='micro')
print("P-micro={0}, R-micro={1}, F1-micro={2}".format(precision_micro, recall_micro, fm_micro))
print("======================================================")
print(metrics.classification_report(y_test, predicted, digits=4))

Accuracy: 0.7997142857142857
P-macro=0.7228835805630008, R-macro=0.7106968771760673, F1-macro=0.7162795301723162
P-micro=0.7997142857142857, R-micro=0.7997142857142857, F1-micro=0.7997142857142857
              precision    recall  f1-score   support

  not sexist     0.8599    0.8806    0.8701      5334
      sexist     0.5858    0.5408    0.5624      1666

    accuracy                         0.7997      7000
   macro avg     0.7229    0.7107    0.7163      7000
weighted avg     0.7947    0.7997    0.7969      7000

